In [64]:
# Find the CSV, load it, and print a quick preview

import os
import re, unicodedata, pandas as pd
from datetime import date

# Discover CSVs under /kaggle/input
csv_paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.lower().endswith('.csv'):
            csv_paths.append(os.path.join(dirname, filename))

if not csv_paths:
    raise FileNotFoundError("No CSV files found under /kaggle/input. Double-check your dataset reference.")

# If there are multiple, just take the first for now (you can pick a specific one if needed)
csv_path = csv_paths[0]
print("Using CSV:", csv_path)

# Load (utf-8-sig handles BOMs from Excel exports nicely)
df = pd.read_csv(csv_path, encoding="utf-8-sig", low_memory=False)

print("Shape:", df.shape)
print("Columns:", list(df.columns))


Using CSV: /kaggle/input/psm-members/members.csv
Shape: (1515, 14)
Columns: ['Submission Time', 'First Name', 'Last Name', 'Birthday', 'E-mail', 'Phone Number', 'Select an Address', 'I understand that all personal data is securely stored on the servers of the Polish Youth Association and is not shared with other institutions.', 'I agree to be kept informed about updates from the Polish Youth Association, including events, scholarships, and the latest news related to youth.', 'Adres korespondencyjny', 'ID', 'Owner', 'Created Date', 'Updated Date']


In [65]:
# Keep only the requested columns (exact header names)
KEEP_COLS = [
    "First Name",
    "Last Name",
    "Birthday",
    "E-mail",
    "Phone Number",
    "Select an Address",
    "Adres korespondencyjny",
]

missing = [c for c in KEEP_COLS if c not in df.columns]
if missing:
    print("WARNING: Missing columns:", missing)

existing = [c for c in KEEP_COLS if c in df.columns]
df_trim = df[existing].copy()

print("Trimmed shape:", df_trim.shape)
print("Trimmed columns:", list(df_trim.columns))


Trimmed shape: (1515, 7)
Trimmed columns: ['First Name', 'Last Name', 'Birthday', 'E-mail', 'Phone Number', 'Select an Address', 'Adres korespondencyjny']


In [66]:
SAA = "Select an Address"
ADR = "Adres korespondencyjny"

# Normalize whitespace / empties
for col in [SAA, ADR]:
    if col in df_trim.columns:
        df_trim[col] = (
            df_trim[col]
            .astype(str)
            .str.strip()
            .replace({"": pd.NA, "nan": pd.NA, "NaN": pd.NA})
        )

# Fill Select an Address from Adres korespondencyjny where SAA is empty and ADR has a value
mask = df_trim[SAA].isna() & df_trim[ADR].notna()
filled_count = int(mask.sum())
df_trim.loc[mask, SAA] = df_trim.loc[mask, ADR]

print(f"Filled {filled_count} empty '{SAA}' cells from '{ADR}'.")

Filled 574 empty 'Select an Address' cells from 'Adres korespondencyjny'.


In [67]:
def nonempty(series: pd.Series) -> pd.Series:
    return (
        series.astype(str)
        .str.strip()
        .replace({"": pd.NA, "nan": pd.NA, "NaN": pd.NA})
        .notna()
    )

# 1) Totals (post-sync)
total_members = len(df_trim)
after_filled = int(nonempty(df_trim[SAA]).sum())

print(f"Total members (rows): {total_members}")
print(f"Members with '{SAA}' filled AFTER sync: {after_filled} ({after_filled/total_members:.1%})")

Total members (rows): 1515
Members with 'Select an Address' filled AFTER sync: 1515 (100.0%)


In [68]:
EMAIL_COL = "E-mail"  # exact header in your CSV

def dup_summary(series: pd.Series):
    s = series.fillna("").astype(str).str.strip()
    # counts per key
    vc = s[s != ""].value_counts()
    dupe_keys = vc[vc > 1]
    rows_in_dupes = int(s.isin(dupe_keys.index).sum())
    unique_nonempty = s.replace("", pd.NA).nunique(dropna=True)
    return unique_nonempty, dupe_keys, rows_in_dupes

# Raw (case-sensitive)
raw_unique, raw_dupe_keys, raw_rows_in_dupes = dup_summary(df_trim[EMAIL_COL])

# Case-insensitive (lowercased)
lower_unique, lower_dupe_keys, lower_rows_in_dupes = dup_summary(df_trim[EMAIL_COL].str.lower())

total = len(df_trim)

print("=== Email duplicate check ===")
print(f"Total rows: {total}")
print("\n-- Exact match (case-sensitive) --")
print(f"Unique emails: {raw_unique}")
print(f"Duplicate email KEYS (count>1): {len(raw_dupe_keys)}")
print(f"Rows that belong to duplicate groups: {raw_rows_in_dupes}")

print("\n-- Case-insensitive (lowercased) --")
print(f"Unique emails: {lower_unique}")
print(f"Duplicate email KEYS (count>1): {len(lower_dupe_keys)}")
print(f"Rows that belong to duplicate groups: {lower_rows_in_dupes}")

=== Email duplicate check ===
Total rows: 1515

-- Exact match (case-sensitive) --
Unique emails: 1286
Duplicate email KEYS (count>1): 174
Rows that belong to duplicate groups: 402

-- Case-insensitive (lowercased) --
Unique emails: 1282
Duplicate email KEYS (count>1): 174
Rows that belong to duplicate groups: 406


In [69]:
EMAIL = "E-mail"
BDAY  = "Birthday"

# 1) Normalize keys for grouping
df_dup = df_trim.copy()
df_dup["_email_lower"] = df_dup[EMAIL].astype(str).str.strip().str.lower()
df_dup["_bday_dt"] = pd.to_datetime(df_dup[BDAY], errors="coerce").dt.date  # date only

# 2) Keep only rows with a non-empty email
df_dup = df_dup[df_dup["_email_lower"].str.len() > 0]

# 3) Consider only emails that appear more than once (case-insensitive)
grp = df_dup.groupby("_email_lower", dropna=False)
dup_emails = grp.size()
dup_emails = dup_emails[dup_emails > 1].sort_values(ascending=False)

print(f"Total rows: {len(df_trim)}")
print(f"Duplicate emails (case-insensitive): {len(dup_emails)}")

# 4) Summarize birthdays within each duplicate email group
def summarize_group(g):
    # unique non-null birthdays
    uniq_bdays = sorted({d for d in g["_bday_dt"].dropna().tolist()})
    n_nonnull  = len(uniq_bdays)
    status = (
        "conflict_birthdays" if n_nonnull > 1 else
        ("same_birthday" if n_nonnull == 1 else "no_birthday_data")
    )
    return pd.Series({
        "rows_in_group": len(g),
        "distinct_birthdays": n_nonnull,
        "birthdays_list": ", ".join(map(str, uniq_bdays)) if uniq_bdays else "",
        "status": status,
    })

dup_summary = (
    df_dup[df_dup["_email_lower"].isin(dup_emails.index)]
    .groupby("_email_lower")
    .apply(summarize_group)
    .reset_index()
    .rename(columns={"_email_lower": "email_lower"})
    .sort_values(["status", "rows_in_group"], ascending=[True, False])
)

# 5) Focused views you’ll care about:
conflicts = dup_summary[dup_summary["status"] == "conflict_birthdays"]
same_bday = dup_summary[dup_summary["status"] == "same_birthday"]
no_bday   = dup_summary[dup_summary["status"] == "no_birthday_data"]

print(f"\nGroups with conflicting birthdays: {len(conflicts)}")
print(f"Groups with the same (non-null) birthday: {len(same_bday)}")
print(f"Groups with no birthday data: {len(no_bday)}")

Total rows: 1515
Duplicate emails (case-insensitive): 174

Groups with conflicting birthdays: 35
Groups with the same (non-null) birthday: 139
Groups with no birthday data: 0


/tmp/ipykernel_36/3822156598.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_group)


In [70]:
EMAIL = "E-mail"
FNAME = "First Name"
LNAME = "Last Name"
BDAY  = "Birthday"
PHONE = "Phone Number"

def ascii_fold(s: str) -> str:
    s = ("" if s is None else str(s)).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # strip diacritics (Michał -> michal)
    s = re.sub(r"[^a-z\s'-]", " ", s)  # keep letters, space, ', -
    s = re.sub(r"\s+", " ", s).strip()
    return s

def phone_digits(s: str) -> str:
    return re.sub(r"\D+", "", "" if s is None else str(s))

# Work copy
d = df_trim.copy()

# Normalizations
d["_email_lower"]  = d[EMAIL].astype(str).str.strip().str.lower()
d["_first_norm"]   = d[FNAME].map(ascii_fold)
d["_last_norm"]    = d[LNAME].map(ascii_fold)
d["_phone_norm"]   = d[PHONE].map(phone_digits) if PHONE in d.columns else ""
d["_dob"]          = pd.to_datetime(d[BDAY], errors="coerce").dt.date

# Build a conservative person key:
#   Prefer (last, first, dob); else (last, first, phone>=7d); else (last, first) and mark for review.
def make_person_key(row):
    if pd.notna(row["_dob"]):
        return f"{row['_last_norm']}|{row['_first_norm']}|{row['_dob']}"
    if row["_phone_norm"] and len(row["_phone_norm"]) >= 7:
        return f"{row['_last_norm']}|{row['_first_norm']}|{row['_phone_norm']}"
    return f"{row['_last_norm']}|{row['_first_norm']}"

d["_person_key"] = d.apply(make_person_key, axis=1)
d["_has_strong_key"] = d["_person_key"].str.contains(r"\|\d{4}-\d{2}-\d{2}$") | d["_person_key"].str.contains(r"\|\d{7,}$")

# Consider only non-empty emails
d = d[d["_email_lower"].str.len() > 0]

# Emails that appear >1 (shared)
vc_email = d["_email_lower"].value_counts()
shared_email_keys = vc_email[vc_email > 1].index

shared = d[d["_email_lower"].isin(shared_email_keys)].copy()

# Summary per email: how many distinct people under that email (by our person_key)
email_person_summary = (
    shared.groupby("_email_lower")
    .agg(
        rows_in_group = (EMAIL, "size"),
        distinct_people = ("_person_key", "nunique"),
        strong_keys = ("_has_strong_key", "sum")
    )
    .sort_values(["distinct_people", "rows_in_group"], ascending=[False, False])
    .reset_index()
)

In [71]:
EMAIL = "E-mail"
FNAME = "First Name"
LNAME = "Last Name"
BDAY  = "Birthday"
PHONE = "Phone Number"
SAA   = "Select an Address"
ADR   = "Adres korespondencyjny"

def ascii_fold(s: str) -> str:
    s = ("" if s is None else str(s)).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # strip diacritics
    s = re.sub(r"[^a-z\s'-]", " ", s)  # keep letters, space, ', -
    s = re.sub(r"\s+", " ", s).strip()
    return s

def phone_digits(s: str) -> str:
    return re.sub(r"\D+", "", "" if s is None else str(s))

def nonempty_count(row, cols):
    cnt = 0
    for c in cols:
        v = row.get(c, None)
        if pd.notna(v) and str(v).strip() != "":
            cnt += 1
    return cnt

def choose_best(grp, value_cols):
    """Pick the 'golden' row: most non-empty fields; then first-in-order tie-breaker."""
    g = grp.copy()
    g["_comp"] = g.apply(lambda r: nonempty_count(r, value_cols), axis=1)
    g = g.sort_values(by=["_comp"], ascending=[False], kind="stable")
    return g.iloc[0]

# Work copy + normalizations
d = df_trim.copy()
d["_email_lower"] = d[EMAIL].astype(str).str.strip().str.lower()
d["_first_norm"]  = d[FNAME].map(ascii_fold)
d["_last_norm"]   = d[LNAME].map(ascii_fold)
d["_dob"]         = pd.to_datetime(d[BDAY], errors="coerce").dt.date
d["_phone_norm"]  = d[PHONE].map(phone_digits) if PHONE in d.columns else ""

# --- Step 1: within (email, name), drop older birthdays; keep newest DOB only ---
value_cols = [FNAME, LNAME, BDAY, EMAIL, PHONE, SAA, ADR]
groups_en = d.groupby(["_email_lower","_last_norm","_first_norm"], dropna=False)

rows_after_dob = []
removed_older_dobs = 0

for key, grp in groups_en:
    # newest DOB within this email+name group
    newest = grp["_dob"].dropna().max() if grp["_dob"].notna().any() else pd.NaT

    if pd.isna(newest):
        # no DOBs in this group -> keep one best row; duplicates will be handled by person_key next
        rows_after_dob.append(choose_best(grp, value_cols))
    else:
        # keep only rows that match the newest DOB
        keep = grp[grp["_dob"] == newest]
        removed_older_dobs += (len(grp) - len(keep))
        # if multiple with same newest DOB, pick the best one here
        rows_after_dob.append(choose_best(keep, value_cols))

d1 = pd.DataFrame(rows_after_dob).reset_index(drop=True)

# --- Step 2: one row per person_key globally (not just within an email) ---
def make_person_key(row):
    # Prefer DOB when present; else phone>=7 digits; else name only
    if pd.notna(row["_dob"]):
        return f"{row['_last_norm']}|{row['_first_norm']}|{row['_dob']}"
    if row["_phone_norm"] and len(row["_phone_norm"]) >= 7:
        return f"{row['_last_norm']}|{row['_first_norm']}|{row['_phone_norm']}"
    return f"{row['_last_norm']}|{row['_first_norm']}"

d1["_person_key"] = d1.apply(make_person_key, axis=1)

rows_final = []
for pk, grp in d1.groupby("_person_key", dropna=False):
    rows_final.append(choose_best(grp, value_cols))

members_clean = pd.DataFrame(rows_final).reset_index(drop=True)

# --- Summaries ---
print("Original rows:", len(df_trim))
print("After dropping older DOBs within same email+name:", len(d1), f"(removed {removed_older_dobs})")
print("Final distinct persons (one row/person):", len(members_clean))
print("Emails that remain shared (multiple persons per email):",
      int(members_clean["_email_lower"].value_counts().gt(1).sum()))

Original rows: 1515
After dropping older DOBs within same email+name: 1302 (removed 30)
Final distinct persons (one row/person): 1276
Emails that remain shared (multiple persons per email): 15


In [72]:
SAA = "Select an Address"
ADR = "Adres korespondencyjny"
EXPORT_PATH = "/kaggle/working/source_of_truth.csv"

# 1) choose the dataset to export
base = members_clean if 'members_clean' in globals() else df_trim.copy()
out = base.copy()

# 2) drop the ADR column
if ADR in out.columns:
    out.drop(columns=[ADR], inplace=True)

# 3) drop helper/engineered columns (anything starting with "_")
helper_cols = [c for c in out.columns if c.startswith("_")]
out.drop(columns=helper_cols, inplace=True, errors="ignore")

# 4) put key columns up front; keep any remaining columns afterward
front = ["First Name", "Last Name", "Birthday", "E-mail", "Phone Number", SAA]
front_existing = [c for c in front if c in out.columns]
out = out[front_existing + [c for c in out.columns if c not in front_existing]]

# 5) write the CSV
out.to_csv(EXPORT_PATH, index=False, encoding="utf-8-sig")

print(f"Wrote: {EXPORT_PATH}")
print(f"Rows: {len(out)} | Columns: {len(out.columns)}")


Wrote: /kaggle/working/source_of_truth.csv
Rows: 1276 | Columns: 6
